# This notebook will be mainly used for the capstone project.

In [1]:
# Downloading the dependencies that will be required
import pandas as pd
import numpy as np  # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
address = 'Gothenburg, Sweden'

geolocator = Nominatim(user_agent="gbg_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Gothenburg are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Gothenburg are 57.7072326, 11.9670171.


In [7]:
# Creating a map of GBG using latitude and longitude values
map_gbg = folium.Map(location=[latitude, longitude], zoom_start=11)
map_gbg

In [9]:
# My Foursquare credentials were stored locally
config = json.load(open('C:/Users/lcuzacov/Desktop/GitHub/Coursera_Capstone/config.json'))
CLIENT_ID = config['CLIENT_ID']
CLIENT_SECRET = config['CLIENT_SECRET']
VERSION = config['VERSION']
radius = 500
LIMIT=100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=MO1YWQXW5UFM555ZUBCSARHPDJFJRDLQWT5ZEOVTVAR34PBH&client_secret=BDCME5RTHIGNBBSCBRS3HMZA3BT3O2TCW3FQR4L1UI2IO0BU&ll=57.7072326,11.9670171&v=20180605&radius=500&limit=100'

In [10]:
results = requests.get(url).json()
results

: '4bf58dd8d48988d16c941735',
         'name': 'Burger Joint',
         'pluralName': 'Burger Joints',
         'shortName': 'Burgers',
         'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/burger_',
          'suffix': '.png'},
         'primary': True}],
       'photos': {'count': 0, 'groups': []}},
      'referralId': 'e-0-4db0468a5da32cf2df45fc16-41'},
     {'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5290f83711d2c11fbbb9c102',
       'name': 'Basque',
       'location': {'address': 'Magasinsgatan 3',
        'crossStreet': 'Makarna Tods Plats',
        'lat': 57.70459637107613,
        'lng': 11.96189212186686,
        'labeledLatLngs': [{'label': 'display',
          'lat': 57.70459637107613,
          'lng': 11.96189212186686}],
        'distance': 423,
        'cc': 'SE',
        'city': 'Göteborg',
        'state': 'Västra 

In [11]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [14]:
venues = results['response']['groups'][0]['items']
    
gbg_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
gbg_venues =gbg_venues.loc[:, filtered_columns]

# filter the category for each row
gbg_venues['venue.categories'] = gbg_venues.apply(get_category_type, axis=1)

# clean columns
gbg_venues.columns = [col.split(".")[-1] for col in gbg_venues.columns]

gbg_venues.head()

,name,categories,lat,lng
0,Hotel Pigalle,Hotel,57.706979,11.970772
1,Tugg,Burger Joint,57.705397,11.965740
2,Ahlströms Konditori,Café,57.705884,11.965707
3,Bar Centro,Coffee Shop,57.705536,11.969646
4,Bhoga,Scandinavian Restaurant,57.706148,11.962454


In [15]:
print('{} venues were returned by Foursquare.'.format(gbg_venues.shape[0]))

59 venues were returned by Foursquare.


In [17]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) 

# add the Italian restaurants as blue circle markers
for lat, lng, label in zip(gbg_venues.lat, gbg_venues.lng, gbg_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map